![Verne](https://www.vernegroup.com/wp-content/uploads/2020/07/LOGO-VERNE-TECHNOLOGY-GROUP-3.png)

# Selección de Características

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect('Tiendas24H.sqlite')
#Consideramos que un cliente nos ha abandonado si no nos compra desde hace más de 200 días. 
consulta = "select c.CodCliente, max(v.fecha) as FechaUltimaCompra, EstadoCivil, Sexo,IngresosAnuales, N.NivelAcademico, O.Ocupacion,  round((julianday('now') - julianday(FechaNacimiento))/365) as Edad,round((julianday('now') - julianday(max(v.fecha)))) as DiasUltimaCompra, case when round((julianday('now') - julianday(max(v.fecha))))>200 then 1 else 0 end as 'Abandono' from clientes c inner join NivelAcademico N  on c.NivelAcademico = N.Codigo  inner join Ocupacion O on c.Ocupacion = o.Codigo inner join VentasCAB v on v.codcliente = C.codcliente where c.Codcliente <>'CONTADO' group by c.CodCliente,  EstadoCivil, Sexo, IngresosAnuales, N.NivelAcademico, O.Ocupacion"

df = pd.read_sql_query(consulta,conn)

In [3]:
df.dropna(axis=0,inplace=True)

In [4]:
#Personalizado
etiquetas = ["Adolescentes", "Millenials" ,"EGB","Maduros"]
custombins = [16,29,39,54,88]
df["RangoEdad"] = pd.cut(df["Edad"],bins=custombins,labels=etiquetas)

In [5]:
caracteristicas=["EstadoCivil" , "Sexo","IngresosAnuales","NivelAcademico","Ocupacion","DiasUltimaCompra","RangoEdad"]
etiqueta = ["Abandono"]

In [6]:
from sklearn.preprocessing import  LabelEncoder

le=LabelEncoder()
df['EstadoCivil'] = le.fit_transform(df['EstadoCivil'])
df['Sexo'] = le.fit_transform(df['Sexo'])

In [7]:
dum_df = pd.get_dummies(df[caracteristicas], columns=["NivelAcademico","Ocupacion","RangoEdad"] )


In [8]:
X, y = dum_df, df[etiqueta]

In [9]:
from imblearn.over_sampling import SMOTE 
from collections import Counter

print('Distribución Original %s' % Counter(y))
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print('Distribución balanceada %s' % Counter(y_res))

Distribución Original Counter({'Abandono': 1})
Distribución balanceada Counter({'Abandono': 1})


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, random_state=1 , test_size=0.3)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 1.00
Accuracy of Decision Tree classifier on test set: 1.00


In [ ]:
# define custom class to fix bug in xgboost 1.0.2
class MyXGBClassifier(XGBClassifier):
	@property
	def coef_(self):
		return None

In [21]:
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel


model = MyXGBClassifier()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Fit model using each importance as a threshold



Accuracy: 100.00%


In [22]:
thresholds = sort(model.feature_importances_)
print (thresholds)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [23]:

for thresh in thresholds:
	# seleccionar caracaterísticas según el umbral
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# entrenar modelo
	selection_model = MyXGBClassifier(XGBClassifier)
	selection_model.fit(select_X_train, y_train)
	# evaluar modelo
	select_X_test = selection.transform(X_test)
	y_pred = selection_model.predict(select_X_test)
	predictions = [round(value) for value in y_pred]
	accuracy = accuracy_score(y_test, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

TypeError: get_params() missing 1 required positional argument: 'self'